<a href="https://colab.research.google.com/github/ramirogalvez/colab_notebooks/blob/master/Ejemplo_de_Sentiment_Analysis_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import zipfile
import io
import re
import pandas as pd
from collections import Counter
from sklearn.ensemble import RandomForestClassifier

# ¿Qué tipos de análisis se pueden hacer programando?

El objetivo de esta notebook es mostrarles un ejemplo del tipo de análisis avanzado que puede hacerse tras aprender a programar. Concretamente, en esta notebook se aborda un problema que ya vimos en clases: **de analizar sentimiento en textos**.

Para abordar este problema usaremos *Python*, un lenguage de programación de alto nivel ampliamente usado en *analytics*. Es el lenguaje que se ve en MAAN II. Un lenguage alternativo (aunque principalmente orientado a Data Science) es *R*, que se ve en Intro a Data Science. Ambas son materias del **campo menor en teconología y ciencias de datos.**

Antes de pasar a trabajar con datos, veamos un concepto fundamental de programación que usaremos a lo largo de esta notebook: la declaración de funciones.

## Funciones

En las próximas celdas de esta notebook veremos cómo se declaran y usan funciones en *Python*. El concepto de funciones es muy similar al de macros tal cual las venimos usando en MAAN I (con la diferencia de que pueden tomar parámetros de entrada y pueden devolver valores como output).

A continuación, declararemos (crearemos) una función llamada "*funcion_de_prueba*" que toma como argumentos dos números, realiza una operación con ellos, y devuelve el resultado de dicha operación.

In [2]:
def funcion_de_prueba(num1, num2):
    # Primera función de prueba
    resultado_intermedio = min([num1, num2])
    return 3 * resultado_intermedio

Una vez declarada la función, uno puede la puede usar sin problemas.
Veamos ejemplos:

In [3]:
# Primer ejemplo
print(funcion_de_prueba(1, 2))

3


In [4]:
# Segundo ejemplo
print(funcion_de_prueba(100, 2))

6


In [5]:
# Tercer ejemplo
print(funcion_de_prueba(4, 4))

12


En lo que resta de esta notebook usaremos funciones ya implementadas por nosotros o por gente que escribió librerías y las hizo públicas para que terceros las usen (igual a cuando en el editor de VBA debiamos activábamos funciones que nos permitían interactuar con *solver*). También usaremos **estructuras de datos** avanzadas (generalmente dentro de las funciones).

**IMPORTANTE**: el fin de esta clase **NO** es que ustedes dominen todos estos conceptos, sino que vean que sepan que existen y que, quienes estén interesados, sepan que pueden seguir aprendiéndolos dentro de la carrera de Economía Empresarial de UTDT.

## Problema al que nos vamos a enfrentar

Nuestro objetivo es armar un sistema que lea un review/reseña de restaurantes y nos indique si la persona que escribió el review está hablando mal o no de la comida que en el restaurante se vende (noten que este problema pueden aplicarse a otros dominios, e.g.: si se habla bien o mal de un político en Twitter, si se está haciendo un buen o mal review de un producto en Mercado Libre, etc.).

Ya vimos en las clases iniciales de MAAN que esto puede hacerse con diccionarios de palabras (aunque también vimos que este enfoque es muy limitado). Ahora veremos cómo resolver el problema utilizando técnicas de inteligencia artificial! (concretamente, de aprendizaje automático/estadístico).

Supongamos que tenemos "datos" como los siguientes:


![texto alternativo](https://drive.google.com/uc?id=1cZLBHwMJa34gMYjQbRve4qqwcb7xfkwN)

Lo que nosotros haremos serán intentar aprender qué patrones (palabras en este caso) tienen los reviews que hablan mal de las comidas y qué patrones tienen los que no hablan mal.

En vez de buscar esto de manera manual, utilizaremos un modelo de aprendizaje automático para aprender estos patrones a partir de los datos. Concretamente utilizaremos un modelo llamado **Random Forest**.

## Datos que utilizaremos

A continución trabajaremos con datos que se obtuvieron de uno de los portales más populares de reseñas de restaurantes del país. Generalmente, para obtener de manera masiva datos de este estilo se hace uso de *scrapers* (programas que tienen como objetivo recolectar datos de internet --- que también se pueden programar usando *Python* con relativa facilidad ---).

La siguiente función carga 13174 reviews que previamente fueron recolectados y que están guardado en un archivo *.zip* en drive.

In [6]:
def carga_datos_reviews(filepath):

    # Descarga el archivo zip y lo descomprime en el server de Google
    r = requests.get(filepath, stream=True)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()

    # Carga los datos referidos al puntaje asignado
    clase = pd.read_csv("./reviews/classificacion.txt", sep = "\t")

    # Guarda el texto y el puntaje clase de cada review en una lista
    review_data = []
    for i, r in clase.iterrows():
        fpath = "./reviews/corpus/" + r.id_comentario + ".txt"
        with open(fpath) as f:
            review = f.read()
        review_data.append({"clase": r.clase_comentario,
                            "review": review})

    return review_data

A continuación ejecutamos la función y cargamos los datos en memoria RAM (del server de Google)

In [7]:
url = "https://drive.google.com/uc?id=1qR1hBSvzHJUeTumyhqF3CnhbzkCGPLhR"
reviews_data = carga_datos_reviews(url)

Exploremos un poco los datos.

In [8]:
# Cantidad de reviews
print(len(reviews_data))

13174


In [9]:
# Imprimamos la review en la posición 10
print(reviews_data[10])

{'clase': 'Excelente', 'review': 'Muy lindo lugar, buena ambientacion, mesas bien separadas, para destacar la atencion de todo el personal. muy amables y atentos a cualquier requerimiento y por sobre todo excelentes las pastas'}


In [10]:
# Imprimamos el puntaje asignado al review en la posición 10
print(reviews_data[10]["clase"])

Excelente


In [11]:
# Imprimamos el texto escrito del review en la posición 10
print(reviews_data[10]["review"])

Muy lindo lugar, buena ambientacion, mesas bien separadas, para destacar la atencion de todo el personal. muy amables y atentos a cualquier requerimiento y por sobre todo excelentes las pastas


## Procesamiento de los datos

Ahora nos enfocaremos en ver cómo podemos procesar los textos de cada review para que sea "simple" para una computadora aprender patrones a partir de los mismos (esto es un área gigante de estudio y existe toda una disciplina que se enfoca especialmente en estos temas, la misma se llama *procesamiento del lenguaje natural*).

### Bags-of-words model (teoría)

Para una computadora es difícil trabajar con el texto tal cuál nosotros leemos (por qué aun no tienen la capacidad de **entender** qué es lo que hay escrito), pero le resulta muy simple trabajar con otras estructuras que conseeven patrones de los textos.

El camino que ahora seguiremos será procesar los textos de cada reviews manera que conserven información relativa a qué escribió cada persona, pero que también permita a una computadora procesarlos de manera simple. Puntualmente, lo que haremos será representar los reviews/documentos de una manera análoga a como se muestra abajo (Fuente: Jurafsky & Matrtin, 2000).

![texto alternativo](https://drive.google.com/uc?id=1XOwP1TlFMcLRTf_XNX6H6PwFyrkfnSEV)


A esta matriz se la conoce como *term-document matrix*. Lo que hace es simplemente contar cuántas veces aparece cada palabra en cada documento (en el caso de la figura, obras de Shakespeare, en el nuestro serán reviews). Noten que la matriz tendrá tantas filas como palabras distintas haya en los documentos y tantas columnas como documentos haya.

Nosotros usaremos la traspuesta de esta matriz, conocida como *document-term matrix* (para mantener esa idea con la que venimos trabajando desde el comienzo de MAAN de que cada fila es una observación y cada columna hace referencia a mediciones hechas a cada observación).

### Bags-of-words model (implementación)

A continuación se presenta una función que toma como input los reviews cargados en memoría y devuelve dos objetos:

1.   Una matriz (a la que internamente en la función llamamos X) en la cual cada fila representa un review, cada columna una palabra, y cada valor representa las apariciones de cada palabra en cada documento.
2.   Un vector (en realidad una "lista" a la que internamente en la función llamamos y) que indica para cada review si no fue malo (y=1) o si sí lo fue (y=0).

A su vez, considera dos variantes sobre el modelo tradicional de bag-of-words:


1.   Sólo conserva columnas para aquellas palabras que aparecen al menos *min_f* veces en los reviews (en donde *min_f* es un parámetro que se puede modificar).
2.   Los valores internos están expresados en terminos relativos relativos a la suma de cada fila. De modo que en un review dado un valor igual a 0.1 para la palabra *rico* (o la que fuera) indica que en ese review el 10% de las palabras escritas es *rico*.



In [12]:
def to_bag_of_words(reviews_data, min_f):

    # Expresión regular para tokenizar
    regex = re.compile("\w+")

    word_counter = {}
    X_tmp = []
    review = []

    # Para cada review cuanto cuántas veces aparece cada palabra
    for r in reviews_data:
        tokens = regex.findall(r["review"].lower())
        for t in tokens:
            word_counter[t] = word_counter.get(t, 0) + 1
        X_tmp.append(dict(Counter(tokens)))
        review.append(int(r["clase"] != "Malo"))  # Aquí armo el vector y

    # Identifico las palabras que aparecen al menos min_f veces
    freq_tokens = set([e for e in word_counter if word_counter[e] >= min_f])

    # Armo la matriz de bad-of-words con las modificaciones mencionadas
    X = []
    while X_tmp:
        r = X_tmp.pop(0)
        r = {e:r[e] for e in r if e in freq_tokens}
        freq_tmp = sum(r.values())
        X.append({t:(r[t]/freq_tmp) for t in r})
        
    X = pd.DataFrame(X, columns=freq_tokens).fillna(0.0)
    y = review

    return X, y


Ahora usemos esta función con los ya cargados.

In [13]:
X, y = to_bag_of_words(reviews_data, 5)

Exploremos un poco X e y:

In [14]:
# Veamos los primeros 5 valores de y
print(y[:20])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1]


In [15]:
# Veo qué porcentaje de los reviews no fueron malos
print(sum(y)/len(y))

0.9045088811294975


In [16]:
# Veo las primeras 5 filas de la matrix X
X.head()

,42,pseudo,agregado,compensar,comanda,pedirlo,heladas,felicito,chico,estacionamiento,elija,merece,mejoro,placentero,microcentro,criollas,auto,hour,cuchara,esperas,jugar,humita,deberian,tenedor,color,malfatti,salia,deliciosas,volvió,postres,jugoso,sambayon,carozo,accesible,habìa,representa,habra,razon,prolijidad,i,...,aroma,relaciòn,predisposición,pasaban,hoja,scon,desatención,bola,actualizar,tibios,prueba,yogur,mila,espectativas,logradas,sábado,parecida,opino,olor,recomienda,explico,alguien,empleados,listo,super,hijas,bosque,tentadora,unico,corridas,indican,suficiente,tokio,usando,01,auténticamente,nadaban,brule,estábamos,cubierto
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Ye expresamos los datos de una manera tal que pueden introducirse de manera facilmente en modelos de aprendizaje automático.

## Modelo de aprendizaje automático

### Aprendizaje supervisado (mínima intuición)

Cómo ya dijimos, en vez de buscar manera manual qué palabras o combinación de palabras aparecen en reviews malos y no malos, encontraremos estos patrones utilizando un modelo de aprendizaje automático. Esta idea cae dentro de lo que se conoce como **aprendizaje supervisado** (temas que ven en Intro a Data Science).

A grandes razgos el esquema que siguen esta familia de modelos es el siguiente (Fuente: Steinbach, Tan & Kumar, 2005):


![texto alternativo](https://drive.google.com/uc?id=1U7192Mf2PF3Lkcu-MgziTejdRMofMATa)

Nuestro equivalente al *Training Set* del esquema presentado es la matriz X y el vector y, a partir de un modelo aprenderá los patrones relevantes para clasificar de manera automática nuevos reviews que aun no vimos.

### Aprendizaje supervisado (en la práctica)

Lo siguiente función toma como input nuestra matriz X y nuesto vector y, y con ellos entrena un modelo conocido como Random Forest.

In [17]:
def train_learning_model(X, y):
    clf = RandomForestClassifier(n_jobs=-1)
    clf.fit(X, y)
    clf.feature_names = list(X.columns.values)
    return clf

Ya tenemos los datos y propusimos un modelos, ahora entrenemos nuestro modelo!

In [18]:
sentiment_analyzer = train_learning_model(X, y)

Listo! Ya tenemos nuestro modelo entrenado. Ahora lo podemos usar para predecir qué dicen reviews que nunca vió.

## Probemos cómo anda nuestro modelo en reviews nuevos

La siguiente función toma input un review y un modelo entrenado, procesa el review para que tenga una estructura acorde a la matriz X y se la pasa al modelo entrenado para que prediga si es una review que está hablando bien o mal de un resturante.

In [19]:
def analyze_new_review(new_review, clf):

    # Pasamos el review al formato de X
    bow = dict(Counter(re.compile("\w+").findall(new_review.lower())))
    bow = {e:bow[e] for e in bow if e in clf.feature_names}
    sum_tokens = sum(bow.values())
    bow = {e:(bow[e]/sum_tokens) for e in bow}
    X_eval = pd.DataFrame([bow], columns = clf.feature_names).fillna(0)

    # Predecimos utilizando nuestro modelos entrenado
    prediction = clf.predict_proba(X_eval)[0, 1]

    return "Buen review :)" if prediction > 0.9 else "Mal review :("

Con algunos ejemplos probemos cómo anda nuestra función.

In [20]:
new_review = """Vine a cenar y me pedí las pastas con frutos de mar, primer
                crítica te vienen 2 langostinos solos con algunas pocas cosas
                más, le pedí si le podía agregar grama a las pastas, me avisaron
                que tenía un costo adicional,pero nunca imaginé que ese costó
                sería equivalente a un plato de pastas me cobraron $ 225 por un
                poco de crema, ni siquiera era abundante, la verdad es un robo.
                Lo único bueno es la atención de los mozos el resto un desastre
                no vuelvo nunca más!!!!"""

print(analyze_new_review(new_review, sentiment_analyzer))

Mal review :(


In [21]:
new_review = """Buenísimo! Pedí en cuarentena y todo super bien.
                Las quesadillas de lomo y pollo geniales, abundantes y muy
                ricas. Las papas con cheddar muy buenas tambien! Recomiendo!"""

print(analyze_new_review(new_review, sentiment_analyzer))

Buen review :)


In [22]:
new_review = """La atención es pésima. Los mozos y mozas no prestan atención a
                nada. Tardaron 30 minutos en traerme la carta y 25 minutos en
                traerme la cuenta. No están capacitados para atender gente
                tienen pésimos modales y todos cara de ORTO. No vuelvo más"""

print(analyze_new_review(new_review, sentiment_analyzer))

Mal review :(


In [23]:
new_review = """En plena cuarentena, el Locro que hicieron para el 1ro. de Mayo
                espectacular, bien completo y generoso a muy buen precio!!!
                Los felicito y esperamos el del 25!!! A no aflojar y ayudar a
                los bodegones de barrio!!"""

print(analyze_new_review(new_review, sentiment_analyzer))

Buen review :)


In [24]:
new_review = """LLAMO POR UN PEDIDO DE HACE DOS HORAS QUE ESTOY ESPERANDO Y NADIE
                SABE NADA, VUELVO Y LLAMO Y ME DICEN QUE TIENEN DEMORA DE
                30minutos, OSEA DOS HORAS Y MEDIAS. Y CUANDO LLEGA ES UNA REAL
                PORGONA TODO, LA VERDAD COMO SE NOTA QUE ESTAN INDRUSTIALIZADOS
                Y NO LES IMPORTA PERDER CLIENTES, PERO LO QUE NO SABEN ES QUE EL
                BOCA A BOCA ES LO PEOR Y TARDE O TEMPRANO VAN A CAER"""

print(analyze_new_review(new_review, sentiment_analyzer))

Mal review :(


In [25]:
new_review = """Hace varios años que voy a este local y me gusta todo en
                general. Buena atención de parte de los chicos y chicas que
                trabajan. Muy limpio."""

print(analyze_new_review(new_review, sentiment_analyzer))

Buen review :)


In [26]:
new_review = """Hay un solo pibe en el café que te toma el pedido, te cobra, lo
                prepara, levanta y limpia las mesas y también lava las tazas
                sucias.  Dejen de explotar adolescentes! En promedio 30 minutos
                de espera por un café con leche..."""

print(analyze_new_review(new_review, sentiment_analyzer))

Mal review :(


In [ ]:
# Escriban un review inventado por ustedes y prueben qué predice el modelo

my_review = """ESCRIBAN UN REVIEW"""

print(analyze_new_review(my_review, sentiment_analyzer))